In [1]:
def record_factory(cls_name, field_names):
    try:
        field_names = field_names.replace(',', ' ').split() #1
    except AttributeError: # 不能调用.replace或.split方法
        pass # 假定field_names本就是标识符组成的序列
    field_names = tuple(field_names) #2
    
    def __init__(self, *args, **kwargs):
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)
        for name, value in attrs.items():
            setattr(self, name, value)
            
    def __iter__(self): #4
        for name in self.__slots__:
            yield getattr(self, name)
            
    def __repr__(self): #5
        values = ', '.join('{}={!r}'.format(*i) for i in zip(self.__slots__, self))
        return '{}({})'.format(self.__class__.__name__, values)
    
    cls_attrs = dict(
        __slots__ = field_names,
        __init__ = __init__,
        __iter__ = __iter__,
        __repr__ = __repr
    ) #6
    
    return type(cls_name, (object,), cls_attrs) #7

In [2]:
MyClass = type('MyClass', (MySuperClass, MyMixin), {'x': 42, 'x2': lambda self: self.x * 2})

NameError: name 'MySuperClass' is not defined

In [1]:
def entity(cls): #1
    for key, attr in cls.__dict__.items(): #2
        if isinstance(attr, Validated): #4
            type_name = type(attr).__name__
            attr.strorage_name = '_{}#{}'.format(type_name, key) #4
    return cls #5

In [1]:
import model_v6 as model
from model_v6 import entity

@entity
class LineItem:
    description = model.NonBlank()
    weight = model.Quantity()
    price = model.Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [3]:
raisins = LineItem('Golden raisins', 10, 6.95)

In [4]:
dir(raisins)[:3]

['_NonBlank#1', '_Quantity#2', '_Quantity#3']

In [5]:
import evaltime

ModuleNotFoundError: No module named 'evaltime'

evalsupport.py：evaltime.py 导入的模块

In [6]:
print('<[100]> evalsupport module start')

def deco_alpha(cls):
    print('<[200]> deco_alpha')
    def inner_1(self):
        print('<[300]> deco_alpha:inner_1')
        
    cls.method_y = inner_1
    return cls

class MetaAleph(type):
    print('<[400]> MetaAleph body')
    
    def __init__(cls, name, bases, dic):
        print('<[500]> MetaAleph.__init__')
        
        def inner_2(self):
            print('<[600]> MetaAleph.__init__:init_2')
            
        cls.method_z = inner_2
        
print('<[700]> evalsupport module end')

<[100]> evalsupport module start
<[400]> MetaAleph body
<[700]> evalsupport module end


evaltime.py：按顺序写出输出的序号标记 <[N]>

In [ ]:
from evalsupport import deco_alpha

print('<[1]> evaltime module start')


class ClassOne():
    print('<[2]> ClassOne body')
    
    